In [1]:
import pandas as pd
import plotly.express as px

model = "deepseek_chat"

predictions = pd.read_parquet(f"output/predictions_{model}.parquet")
predictions["vote_correct"] = predictions["prediction"] == predictions["ground_truth"]
predictions = predictions[predictions["ground_truth"].notna()]
predictions.drop_duplicates(subset=["vote_id", "drucksache_id", "party"], inplace=True)

In [2]:
import pandas as pd
import plotly.express as px

df = predictions.groupby("is_governing")["vote_correct"].mean().reset_index()
df["Status"] = df["is_governing"].map({True: "Regierung", False: "Opposition"})
df["Percent"] = (df["vote_correct"] * 100).round(1).astype(str) + "%"

fig = px.bar(
    df,
    x="Status",
    y="vote_correct",
    text="Percent",
    color="Status",
    color_discrete_map={"Regierung":"#1f77b4","Opposition":"#ff7f0e"}
)

fig.update_layout(
    yaxis_title="Programmkonsistenz",
    yaxis_tickformat=".0%",
    xaxis_title="",
    showlegend=False,
    title="Durchschnittliche Programmkonsistenz: Regierung vs. Opposition"
)

fig.show()


In [3]:
party_line_correct_plot_data = predictions.groupby("party")["vote_correct"].mean()
party_line_correct_plot_data["average"] = predictions["vote_correct"].mean()


df = party_line_correct_plot_data.reset_index().sort_values(by='vote_correct', ascending=False)
df.columns = ['Party', 'Correct']
df['Percent'] = (df['Correct'] * 100).round(1).astype(str) + '%'
color_map = {
    'DIE_LINKE': '#BE3075',
    'AfD': '#00A2DE',
    'Union': '#000000',
    'FDP': '#FFED00',
    'DIE_GRÜNEN': '#409A3C',
    'SPD': '#E3000F',
    'average': '#808080'
}

fig = px.bar(
    df,
    x='Party',
    y='Correct',
    color='Party',
    color_discrete_map=color_map,
    text='Percent'
)

fig.update_layout(
    xaxis_title='Party',
    yaxis_title='Prozent',
    showlegend=False,
    title='Wie oft halten sich die Parteien an das eigene Wahlprogramm?',
    bargap=0.4
)

fig.show()


In [4]:
import pandas as pd
import plotly.graph_objects as go

color_map = {
    'DIE_LINKE': '#BE3075',
    'AfD':       '#00A2DE',
    'Union':     '#000000',
    'FDP':       '#FFED00',
    'DIE_GRÜNEN':'#409A3C',
    'SPD':       '#E3000F'
}

grouped = (
    predictions
    .groupby(['party', 'is_governing'], as_index=False)['vote_correct']
    .mean()
)
grouped = grouped[~grouped["party"].isin(["AfD", "DIE_LINKE"])]
diffs = grouped.pivot(index='party', columns='is_governing', values='vote_correct')
diffs['gap'] = (diffs[True] - diffs[False]).abs()
ordered_parties = diffs.sort_values('gap', ascending=False).index.tolist()

fig = go.Figure()

for party in ordered_parties:
    x_vals = [diffs.loc[party, False], diffs.loc[party, True]]
    fig.add_trace(go.Scatter(
        x=x_vals,
        y=[party, party],
        mode='lines',
        line=dict(color=color_map[party]),
        showlegend=False,
        hoverinfo='none'
    ))

fig.add_trace(go.Scatter(
    x=diffs[True].reindex(ordered_parties),
    y=ordered_parties,
    mode='markers',
    name='Regierung',
    marker=dict(symbol='circle', size=10, color=[color_map[p] for p in ordered_parties]),
    hovertemplate='Partei: %{y}<br>Regierung: %{x:.1%}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=diffs[False].reindex(ordered_parties),
    y=ordered_parties,
    mode='markers',
    name='Opposition',
    marker=dict(symbol='circle-open', size=10, color=[color_map[p] for p in ordered_parties]),
    hovertemplate='Partei: %{y}<br>Opposition: %{x:.1%}<extra></extra>'
))

fig.update_layout(
    title="Regierung vs. Opposition pro Partei",
    xaxis_title="Programmkonsistenz",
    yaxis_title="Partei",
    xaxis_tickformat=".0%",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    margin=dict(l=150, r=50, t=100, b=50)
)

fig.show()


In [5]:
by_category = predictions.groupby("category")["vote_correct"].mean().sort_values()

fig = px.bar(
    by_category,
    x=[cat.split("-")[0].strip() for cat in by_category.index],
    y=by_category.values,
    color=by_category.index,
    text=[f"{v:.2%}" for v in by_category.values],
    labels={"x": "Category", "y": "Manifesto Consistency"},
    title="Manifesto Consistency by Category",
)

fig.update_layout(
    xaxis_title="Category",
    yaxis_title="Manifesto Consistency",
    showlegend=False
)

fig.show()

In [6]:
import plotly.graph_objects as go


df = (
    predictions.groupby(["is_governing", "category"])["vote_correct"]
    .mean()
    .to_frame()
    .reset_index()
    .sort_values(by=["is_governing", "vote_correct"], ascending=[True, False])
)

votes_per_category = predictions.groupby("category")["vote_id"].nunique()
statistically_relevant = votes_per_category[votes_per_category > 10].index
df = df[df["category"].isin(statistically_relevant)]

# assume df is your DataFrame
# sort categories so the largest gaps stand out (optional)
# you could sort by abs difference:
diffs = df.pivot(index="category", columns="is_governing", values="vote_correct")
diffs["gap"] = (diffs[True] - diffs[False]).abs()
ordered_cats = diffs.sort_values("gap", ascending=False).index.tolist()
df["category"] = pd.Categorical(df["category"], categories=ordered_cats, ordered=True)

fig = go.Figure()

# add a line for each category
for cat in ordered_cats:
    sub = df[df["category"] == cat]
    fig.add_trace(
        go.Scatter(
            x=sub["vote_correct"],
            y=[cat, cat],
            mode="lines",
            line=dict(color="lightgray"),
            showlegend=False,
            hoverinfo="none",
        )
    )

# add the two sets of points
fig.add_trace(
    go.Scatter(
        x=df[df["is_governing"] == True]["vote_correct"],
        y=df[df["is_governing"] == True]["category"],
        mode="markers",
        name="Government",
        marker=dict(symbol="circle", size=10),
    )
)
fig.add_trace(
    go.Scatter(
        x=df[df["is_governing"] == False]["vote_correct"],
        y=df[df["is_governing"] == False]["category"],
        mode="markers",
        name="Opposition",
        marker=dict(symbol="circle", size=10),
    )
)

fig.update_layout(
    title="Government vs. Opposition by Category",
    xaxis_title="Manifesto Consistency",
    yaxis_title="Category",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

fig.show()


In [7]:


import plotly.graph_objects as go
from plotly.subplots import make_subplots

only_governing = predictions[predictions["is_governing"]].copy()

grouped = (
    only_governing
    .groupby(["bundestag", "party"])["vote_correct"]
    .mean()
    .reset_index()
)

bundestags = grouped["bundestag"].unique()
n_cols = len(bundestags)

fig = make_subplots(rows=1, cols=n_cols, subplot_titles=[str(b) for b in bundestags])

for i, bundestag in enumerate(bundestags, start=1):
    df_sub = grouped[grouped["bundestag"] == bundestag]
    fig.add_trace(
        go.Bar(
            x=df_sub["party"],
            y=df_sub["vote_correct"],
            marker_color=[color_map.get(p, "#cccccc") for p in df_sub["party"]],
            text=[f"{v:.0%}" for v in df_sub["vote_correct"]],
        ),
        row=1,
        col=i
    )

fig.update_layout(
    title_text="Average Vote Consistency by Party and Bundestag (Only Governing Parties on their own proposals)",
    showlegend=False,
    height=400
)

fig.show()


In [8]:


import plotly.graph_objects as go
from plotly.subplots import make_subplots

grouped = (
    predictions
    .groupby(["bundestag", "party"])["vote_correct"]
    .mean()
    .reset_index()
)

bundestags = grouped["bundestag"].unique()
n_cols = len(bundestags)

fig = make_subplots(rows=1, cols=n_cols, subplot_titles=[str(b) for b in bundestags])

for i, bundestag in enumerate(bundestags, start=1):
    df_sub = grouped[grouped["bundestag"] == bundestag]
    fig.add_trace(
        go.Bar(
            x=df_sub["party"],
            y=df_sub["vote_correct"],
            marker_color=[color_map.get(p, "#cccccc") for p in df_sub["party"]],
            text=[f"{v:.0%}" for v in df_sub["vote_correct"]],
        ),
        row=1,
        col=i
    )

fig.update_layout(
    title_text="Average Vote Consistency by Party and Bundestag",
    showlegend=False,
    height=400
)

fig.show()
